In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
print("Python version")
print (sys.version)

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot
print(kale.__version__)

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Apply population modifiers
redz = cosmo.a_to_z(pop.scafa)

hards_int2d = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(int2d = True),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

hards_rbspl = [
    holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(int2d = False),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

f_edd = 0.1
all_evols = {}
eccen = 0.8

acc_int2d = holo.accretion.Accretion(accmod='Siwek22', f_edd = f_edd, eccen=eccen, subpc=True, int2d = True)
evo_int2d = holo.evolution.Evolution(pop, hards_int2d, nsteps = 100, debug=True, acc=acc_int2d)
evo_int2d.evolve()

acc_rbspl = holo.accretion.Accretion(accmod='Siwek22', f_edd = f_edd, eccen=eccen, subpc=True, int2d = False)
evo_rbspl = holo.evolution.Evolution(pop, hards_rbspl, nsteps = 100, debug=True, acc=acc_rbspl)
evo_rbspl.evolve()

In [ ]:
#Take a few random numbers and plot their mass evolutions as a function of time
import random
fs = 20

n_mbhb = evo_rbspl.shape[0]

n_mbhb_to_plot = 2
inds_random_mbhb = []

p = 0
while p < n_mbhb_to_plot:
    x = random.randint(0,n_mbhb)
    inds_t_pos = evo_rbspl.tlook[x] > 0
    if len(evo_rbspl.tlook[x][inds_t_pos]) > 5:
        inds_random_mbhb.append(x)
        p += 1

print("inds_random_mbhb = ",inds_random_mbhb)

fig, axs = plt.subplots(int(n_mbhb_to_plot),3,figsize=(25,15))
fig.suptitle('Separations and Masses of MBHBs', fontsize=fs)
k = 0
for i in range(0,int(n_mbhb_to_plot)):
    #for j in range(0,int(n_mbhb_to_plot/2)):
    for evo,label in zip([evo_int2d, evo_rbspl], ['int2d', 'rbspl']):
        inds_t_pos = evo.tlook[inds_random_mbhb[k]] > 0
        time = (evo.tlook[inds_random_mbhb[k]][inds_t_pos])
        masses = (evo.mass[inds_random_mbhb[k]][inds_t_pos])
        separation = (evo.sepa[inds_random_mbhb[k]][inds_t_pos])
        dadt = -(evo.dadt[inds_random_mbhb[k]][inds_t_pos])
        #PLOT SEPARATIONS
        j = 0
        axs[i][j].loglog(separation/PC, (separation/dadt)/GYR, '*-', label=label)
        axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
        #axs[i][j].set_ylabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
        axs[i][j].set_ylabel(r'$t_{\rm{coal}} \, [\rm{GYR}]$', fontsize=fs)
        #axs[i][j].invert_xaxis()
        #axs[i][j].legend(fontsize=fs)
        plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
        plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
        #PLOT MASSES
        j = 1
        axs[i][j].loglog((separation/PC), dadt/(PC/GYR), '*-', label = label)
        axs[i][j].set_xlabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
        axs[i][j].set_ylabel(r'$\rm dadt$', fontsize=fs)
        axs[i][j].invert_xaxis()
        axs[i][j].legend(fontsize=fs)
        plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
        plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
        j = 2
        if masses.T[0][0] >= masses.T[1][0]:
            q_b = np.array(masses.T[1])/np.array(masses.T[0])
            #axs[i][j].loglog((separation/PC), masses.T[0]/MSOL, '*-', label = 'Primary')
            #axs[i][j].loglog(separation/PC, masses.T[1]/MSOL, '*-', label = 'Secondary, '+ label)
            axs[i][j].loglog(separation/PC, q_b, '*-', label = r'$q_{\rm b},\ $'+ label)
        else:
            q_b = np.array(masses.T[0])/np.array(masses.T[1])
            #axs[i][j].loglog(separation/PC, masses.T[1]/MSOL, '*-', label = 'Primary')
            #axs[i][j].loglog(separation/PC, masses.T[0]/MSOL, '*-', label = 'Secondary, ' + label)
            axs[i][j].loglog(separation/PC, q_b, '*-', label = r'$q_{\rm b},\ $'+ label)
        #axs[i][j].set_xlabel(r'$t \, [GYR]$', fontsize=fs)
        axs[i][j].set_xlabel(r'$a_{\rm{b}} \, [\rm{PC}]$', fontsize=fs)
        #axs[i][j].set_ylabel(r'$M_{\rm{MBHB}}/M_{\odot}$', fontsize=fs)
        axs[i][j].set_ylabel(r'$q_{\rm{b}}$', fontsize=fs)
        axs[i][j].legend(fontsize=fs)
        plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
        plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
        axs[i][j].invert_xaxis()
        
    k+=1

        
plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])


In [ ]:
import random
import seaborn as sns
palette = sns.color_palette('muted')

all_evols = {
    'int2d': evo_int2d,
    'rbspl': evo_rbspl
    }
run_ids = ['int2d', 'rbspl']

all_colors = {
'int2d': palette[0],
'rbspl': palette[1],
}

all_q = {}

for rid in run_ids:
    evo = all_evols['%s' %rid]
    all_q['q_init_%s' %rid] = []
    all_q['q_final_%s' %rid] = []
    masses = evo.mass
    for i in np.arange(0,np.shape(masses)[0]):
        inds_t_pos = evo.tlook[i] > 0
        masses = (evo.mass[i][inds_t_pos])
        if len(evo.tlook[i][inds_t_pos]) > 5:
            if masses.T[0][0] > masses.T[1][0]:
                primary_init = masses.T[0][0]
                secondary_init = masses.T[1][0]
            else:
                primary_init = masses.T[1][0]
                secondary_init = masses.T[0][0]
            if masses.T[0][-1] > masses.T[1][-1]:
                primary_final = masses.T[0][-1]
                secondary_final = masses.T[1][-1]
            else:
                primary_final = masses.T[1][-1]
                secondary_final = masses.T[0][-1]
            q_init = secondary_init/primary_init
            q_final = secondary_final/primary_final
            if q_init>=0.1:
                all_q['q_init_%s' %rid].append(q_init)
                all_q['q_final_%s' %rid].append(q_final)

fs = 20
nbins = 100
bins = np.linspace(1.e-1, 1.0, nbins)
#logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
logbins = bins
labels = run_ids

fig, axs = plt.subplots(len(all_evols)+1,1,figsize=(15,20))
for ax,evo,label in zip(axs,all_evols,labels):
    ax.hist(all_q['q_init_%s' %label], bins = logbins, log=True, \
            label=r'$q_{\rm init},$' + ' mean=%.2f' %(np.mean(all_q['q_init_%s' %label])), \
            alpha=0.7, color='gray')
    ax.hist(all_q['q_final_%s' %label], bins = logbins, log=True, \
            label=r'$q_{\rm final},$' + ' mean=%.2f' %(np.mean(all_q['q_final_%s' %label])), \
            alpha=0.7, color=all_colors[label])
    ax.legend(fontsize=fs)
    ax.set_title('%s, %d binaries.' %(label,len(all_q['q_final_%s' %label])), fontsize=fs)
    #ax.set_xscale('log')
    plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
    plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
    
    axs[-1].hist(all_q['q_final_%s' %label], bins = logbins, log=True, \
            label='%s, mean=%.2f' %(label,np.mean(all_q['q_final_%s' %label])), \
            alpha=0.7, color=all_colors[label])

axs[-1].legend(fontsize=fs)
plt.setp(axs[-1].get_xticklabels(which='both'), fontsize=fs, rotation=45)
plt.setp(axs[-1].get_yticklabels(which='both'), fontsize=fs)

plt.tight_layout()